In [ ]:
%run main --yaml_path yamls/protein_vec.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_8.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_35.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_150.yaml

In [ ]:
%run main.py --eval --yaml_path yamls/benchmarking/esm_sim_benchmark_650.yaml

In [ ]:
import torch
from transformers import BertConfig, BertTokenizer
from models.model_zoo import *
from utils import get_yaml
from glob import glob

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
yargs = get_yaml('yamls/SE/cvd.yaml')
config = BertConfig.from_pretrained(yargs['general_args']['model_path'])
for key, value in yargs['general_args'].items(): # copy yaml config into args
    setattr(config, key, value)


In [ ]:
weights = [path.replace('\\', '/') for path in glob('./weights/*.pt')]
weights

In [ ]:
paths = []
for weight in weights:
    model = torch.load(weight)
    try:
        model.config = model.bert.config
    except:
        print(weight)
        continue
    path = 'lhallee/' + weight.split('/')[-1].split('.pt')[0]
    paths.append(path)
    #model.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('results/NLP_MOE')

In [ ]:
for path in paths:
    tokenizer.push_to_hub(path)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('lhallee/sci_moe_all')

In [ ]:
tokenizer.push_to_hub('lhallee/all_nlp_singlemoe_MI')

In [1]:
import torch
from models.protein_vec.src_run.huggingface_protein_vec import ProteinVec, ProteinVecConfig

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
disk_model.push_to_hub('lhallee/ProteinVec')

In [2]:
disk_model = ProteinVec(config=ProteinVecConfig())
disk_model.load_from_disk()

Lightning automatically upgraded your loaded checkpoint from v1.8.0rc0 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\lhall\Desktop\Research\Mixture-of-Experts-Sentence-Similarity\models\protein_vec\src_run\protein_vec_models\tm_vec_swiss_model_large.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.9.4 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\lhall\Desktop\Research\Mixture-of-Experts-Sentence-Similarity\models\protein_vec\src_run\protein_vec_models\aspect_vec_pfam.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.9.4 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\lhall\Desktop\Research\Mixture-of-Experts-Sentence-Similarity\models\protein_vec\src_run\protein_vec_models\aspect_v

In [ ]:
test = torch.load('models/protein_vec/src_run/protein_vec_models/protein_vec.ckpt')

In [ ]:
for k, v in test['state_dict'].items():
    print(k)

In [3]:
hf_model = ProteinVec.from_pretrained('lhallee/ProteinVec', config=ProteinVecConfig())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import torch

def are_models_equal(model1, model2):
    state_dict1 = model1.state_dict()
    state_dict2 = model2.state_dict()
    
    if len(state_dict1) != len(state_dict2):
        print('Length')
        return False
    
    for key in state_dict1:
        if key not in state_dict2:
            print('Key')
            return False
        
        if not torch.equal(state_dict1[key], state_dict2[key]):
            print('Equal')
            return False
    
    return True


if are_models_equal(disk_model.cpu(), hf_model.cpu()):
    print("The models are equal.")
else:
    print("The models are not equal.")

The models are equal.


In [ ]:
        #encoder_state_dict = OrderedDict()
        #for key, value in state_dict.items():
        #    if key.startswith('encoder.'):
        #        new_key = key[len('encoder.'):]  # Remove 'encoder.' prefix
        #        encoder_state_dict[new_key] = value
        #    else:
        #        encoder_state_dict[key] = value#
        #self.moe.encoder.load_state_dict(encoder_state_dict)

